In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    r"/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/abi_hackathon_2k24"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/abi_hackathon_2k24


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)
from pandarallel import pandarallel
import re
from datetime import datetime
import gc
from sklearn.preprocessing import FunctionTransformer

# from helpers.compress import reduce_mem_usage
import pickle
import warnings
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
warnings.filterwarnings("ignore")

In [4]:
data_folder = "data"
macro_data_folder = "macro_data"
processed_data_folder = "processed_data_S2"
processed_macro = "macro_processed"
fe_data = "feature_engineered_data"

In [5]:
volume_monthly_df_best = pd.read_pickle(
    os.path.join(
        os.getcwd(), processed_data_folder, "submission_id_vol_cluster_monthly_df_all_feats.pkl"
    )
)

In [6]:
submission_df = pd.read_csv(
    os.path.join(
        os.getcwd(), data_folder, "sample_submission_file.csv"
    )
)

In [7]:
# best_feats_by_last_run = ['sales_roll_mean_9',
#  'sales_roll_mean_3',
#  'sales_roll_mean_6',
#  'sales_ewm_alpha_08_lag_14',
#  'sales_ewm_alpha_08_lag_13',
#  'sales_ewm_alpha_08_lag_12',
#  'sales_ewm_alpha_095_lag_14',
#  'sales_ewm_alpha_095_lag_13',
#  'sales_lag_15',
#  'sales_ewm_alpha_09_lag_12',
#  'sales_ewm_alpha_09_lag_13',
#  'id_life_flag',
#  'sales_ewm_alpha_095_lag_12',
#  'sales_ewm_alpha_09_lag_14',
#  'segment',
#  'sales_lag_14',
#  'ppg_name',
#  'sales_lag_13',
#  'sales_lag_12',
#  'subchannel',
#  'sales_lag_13_diff',
#  'channel_type',
#  'unique_poc_id',
#  'sales_lag_15_diff',
#  'poc_ppg_cluster',
#  'sales_lag_14_diff',
#  'poc_ppg_id',
#  'nr_hl',
#  'maco_hl',
#  'poc_seasonality_index',
#  'brand',
#  'ppg_seasonality_index',
#  'price_segment']+["year_month_start_date","sales_volume_hl_agg"]

In [8]:
test_cut_date = "2023-01-01"
full_train_df = volume_monthly_df_best.loc[volume_monthly_df_best['year_month_start_date']<test_cut_date,].reset_index(drop=True)
test_df =  volume_monthly_df_best.loc[volume_monthly_df_best['year_month_start_date']>=test_cut_date,].reset_index(drop=True)

In [9]:
full_train_df.shape

(1973838, 67)

In [10]:
test_df.shape

(592044, 67)

In [11]:
# test_df['poc_ppg_id'].nunique() *12

In [12]:
val_date = "2022-07-01"
train_df = full_train_df.loc[full_train_df['year_month_start_date']<val_date].reset_index(drop=True)
val_df = full_train_df.loc[full_train_df['year_month_start_date']>=val_date].reset_index(drop=True)

In [13]:
train_df["sales_volume_hl_agg"] = np.log1p(train_df["sales_volume_hl_agg"])
val_df['sales_volume_hl_agg'] = np.log1p(val_df['sales_volume_hl_agg'])

In [14]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common import space

In [15]:
label = 'sales_volume_hl_agg'

In [16]:
y_val = val_df[label]
val_df_nolabel = val_df.drop(columns=[label])

In [17]:
metric='mape'

In [18]:
# GBM,XGB

In [19]:
gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': space.Int(lower=100, upper=1000, default=200), 
    'num_leaves': space.Int(lower=26, upper=66, default=36),
    'learning_rate' : space.Real(0.001, 0.1, log=True)

}
xgb_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'max_depth': space.Int(lower=6, upper=15, default=10),
    'eta' : space.Real(0.001, 0.1, log=True)
}
cb_options = {  # specifies non-default hyperparameter values for neural network models
    "depth": space.Int(lower=1, upper=12, default=5), 
    'learning_rate' : space.Real(0.001, 0.1, log=True),
}

In [ ]:
hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'XGB': xgb_options, 
#                     "CAT":{},
                  }  
num_trials = 15  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}  # Refer to TabularPredictor.fit docstring for all valid values

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_df,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs, 
#     presets="high_quality",
#     refit_full=True

)

No path specified. Models will be saved in: "AutogluonModels/ag-20240719_182724"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #73~20.04.1-Ubuntu SMP Mon May 6 09:43:44 UTC 2024
CPU Count:          16
Memory Avail:       99.23 GB / 108.07 GB (91.8%)
Disk Space Avail:   5026.75 GB / 5120.00 GB (98.2%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality'

  0%|          | 0/15 [00:00<?, ?it/s]

Fitted model: LightGBM/T1 ...
	-1325306063470020.8	 = Validation score   (-mean_absolute_percentage_error)
	34.49s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	-560627246766489.6	 = Validation score   (-mean_absolute_percentage_error)
	160.0s	 = Training   runtime
	0.99s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	-548138762298945.3	 = Validation score   (-mean_absolute_percentage_error)
	234.09s	 = Training   runtime
	1.39s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	-516494500450484.25	 = Validation score   (-mean_absolute_percentage_error)
	217.04s	 = Training   runtime
	1.19s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	-567126306902170.4	 = Validation score   (-mean_absolute_percentage_error)
	175.35s	 = Training   runtime
	0.62s	 = Validation runtime
Fitted model: LightGBM/T6 ...
	-876385165537601.0	 = Validation score   (-mean_absolute_percentage_error)
	124.13s	 = Training   runtime
	0.14s	 = Validation runtime
Fitt

In [20]:
# predictor = TabularPredictor.load('AutogluonModels/ag-20240719_140049')

In [23]:
y_pred = predictor.predict(val_df_nolabel)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(val_df, auxiliary_metrics=False)

Predictions:   [3.4282121658325195, 3.8710477352142334, 3.722357749938965, 3.482429265975952, 5.536045551300049]


In [24]:
predictor.leaderboard(val_df)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM/T4,-0.478273,-0.498304,symmetric_mean_absolute_percentage_error,2.063852,0.676666,118.621577,2.063852,0.676666,118.621577,1,True,4
1,WeightedEnsemble_L2,-0.479375,-0.484791,symmetric_mean_absolute_percentage_error,13.329825,59.059402,855.353684,0.019987,0.000866,0.069898,2,True,32
2,XGBoost/T3,-0.480110,-0.485160,symmetric_mean_absolute_percentage_error,4.322467,0.454958,141.157547,4.322467,0.454958,141.157547,1,True,19
3,XGBoost/T4,-0.480552,-0.486348,symmetric_mean_absolute_percentage_error,1.767745,0.204395,38.535929,1.767745,0.204395,38.535929,1,True,20
4,XGBoost/T7,-0.483502,-0.488049,symmetric_mean_absolute_percentage_error,1.872818,0.215923,40.328414,1.872818,0.215923,40.328414,1,True,23
5,LightGBM/T3,-0.483976,-0.501908,symmetric_mean_absolute_percentage_error,1.925297,0.759296,117.827363,1.925297,0.759296,117.827363,1,True,3
6,ExtraTrees,-0.486731,-0.501386,symmetric_mean_absolute_percentage_error,5.155774,57.722517,556.968734,5.155774,57.722517,556.968734,1,True,16
7,LightGBM/T5,-0.487586,-0.505807,symmetric_mean_absolute_percentage_error,1.567196,0.491575,104.306359,1.567196,0.491575,104.306359,1,True,5
8,LightGBM/T8,-0.488001,-0.504241,symmetric_mean_absolute_percentage_error,1.277880,0.281002,75.258482,1.277880,0.281002,75.258482,1,True,8
9,LightGBM/T10,-0.488111,-0.504939,symmetric_mean_absolute_percentage_error,2.045429,0.570171,126.875598,2.045429,0.570171,126.875598,1,True,10


In [25]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val                               eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  -0.484791  symmetric_mean_absolute_percentage_error      59.059402  855.353684                0.000866           0.069898            2       True         32
1            XGBoost/T3  -0.485160  symmetric_mean_absolute_percentage_error       0.454958  141.157547                0.454958         141.157547            1       True         19
2            XGBoost/T4  -0.486348  symmetric_mean_absolute_percentage_error       0.204395   38.535929                0.204395          38.535929            1       True         20
3            XGBoost/T7  -0.488049  symmetric_mean_absolute_percentage_error       0.215923   40.328414                0.215923          40.328414            1       True         23
4           XGBoost/T13  -0.

In [26]:
predictor.evaluate(val_df)

{'symmetric_mean_absolute_percentage_error': -0.47937503475118065,
 'root_mean_squared_error': -17.401617912758425,
 'mean_squared_error': -302.81630598163486,
 'mean_absolute_error': -5.1891825199249535,
 'r2': 0.9151017091440898,
 'pearsonr': 0.9578939504012386,
 'median_absolute_error': -1.6064415602813082}

In [49]:
feats_df = predictor.feature_importance(val_df,subsample_size=int(len(val_df)/3),num_shuffle_sets=3)

Computing feature importance via permutation shuffling for 66 features using 97137 rows with 3 shuffle sets...
	2333.86s	= Expected runtime (777.95s per shuffle set)
	782.16s	= Actual runtime (Completed 3 of 3 shuffle sets)


In [55]:
feats_df.to_pickle(os.path.join(os.getcwd(),"sample_submissions",'feat_imp_all_feats.pkl'))

In [135]:
feats_df.head(33).index.tolist()

['sales_roll_mean_9',
 'sales_roll_mean_3',
 'sales_roll_mean_6',
 'sales_ewm_alpha_08_lag_14',
 'sales_ewm_alpha_08_lag_13',
 'sales_ewm_alpha_08_lag_12',
 'sales_ewm_alpha_095_lag_14',
 'sales_ewm_alpha_095_lag_13',
 'sales_lag_15',
 'sales_ewm_alpha_09_lag_12',
 'sales_ewm_alpha_09_lag_13',
 'id_life_flag',
 'sales_ewm_alpha_095_lag_12',
 'sales_ewm_alpha_09_lag_14',
 'segment',
 'sales_lag_14',
 'ppg_name',
 'sales_lag_13',
 'sales_lag_12',
 'subchannel',
 'sales_lag_13_diff',
 'channel_type',
 'unique_poc_id',
 'sales_lag_15_diff',
 'poc_ppg_cluster',
 'sales_lag_14_diff',
 'poc_ppg_id',
 'nr_hl',
 'maco_hl',
 'poc_seasonality_index',
 'brand',
 'ppg_seasonality_index',
 'price_segment']

###### Predictions

In [57]:
id_month_vol_bands = pd.read_pickle(
    os.path.join(
        os.getcwd(), processed_data_folder, "id_month_vol_bands.pkl"
    )
)

In [58]:
id_month_vol_bands.head()

,poc_ppg_id,month,low_band,high_band,max_band
0,AAAC__CJ_C_FN_M,1,0.030658,0.610095,0.612547
1,AAAC__CJ_C_FN_M,2,0.000000,0.000000,0.000000
2,AAAC__CJ_C_FN_M,3,0.061316,1.220190,1.225095
3,AAAC__CJ_C_FN_M,4,0.022483,0.447403,0.449201
4,AAAC__CJ_C_FN_M,5,0.265703,5.287488,5.308745


In [88]:
test_df_id = test_df[['poc_ppg_id','ppg_name','unique_poc_id','year','month','quarter']]

In [74]:
y_pred_test = predictor.predict(test_df)

In [89]:
test_df_id['quarterly_sales_hl'] = y_pred_test

In [90]:
test_df_id['quarterly_sales_hl'].mean()

13.372055

In [91]:
test_df_id.columns

Index(['poc_ppg_id', 'ppg_name', 'unique_poc_id', 'year', 'month', 'quarter',
       'quarterly_sales_hl'],
      dtype='object')

In [92]:
test_df_id.shape

(592044, 7)

In [93]:
test_df_id.drop_duplicates(['ppg_name','unique_poc_id','year','month']).shape

(592044, 7)

In [94]:
test_df_id = test_df_id.groupby(['poc_ppg_id','ppg_name','unique_poc_id','year','quarter','month'],as_index=False).agg({"quarterly_sales_hl":"mean"})

In [95]:
test_df_id['quarterly_sales_hl'].mean()

13.372055

In [97]:
test_df_id.shape

(592044, 7)

In [83]:
id_month_vol_bands.head()

,poc_ppg_id,month,low_band,high_band,max_band
0,AAAC__CJ_C_FN_M,1,0.030658,0.610095,0.612547
1,AAAC__CJ_C_FN_M,2,0.000000,0.000000,0.000000
2,AAAC__CJ_C_FN_M,3,0.061316,1.220190,1.225095
3,AAAC__CJ_C_FN_M,4,0.022483,0.447403,0.449201
4,AAAC__CJ_C_FN_M,5,0.265703,5.287488,5.308745


In [99]:
test_df_id = pd.merge(test_df_id,id_month_vol_bands, on =['poc_ppg_id','month'], how='left')

In [102]:
test_df_id.isnull().sum()

poc_ppg_id            0
ppg_name              0
unique_poc_id         0
year                  0
quarter               0
month                 0
quarterly_sales_hl    0
low_band              0
high_band             0
max_band              0
dtype: int64

In [118]:
def adjusted(x):
    if x['quarterly_sales_hl'] <= x['low_band']:
        return x['low_band']
    elif x['quarterly_sales_hl'] >= 1.5 * x['max_band']:
        return x['max_band']
    else:
        return x['quarterly_sales_hl']

In [119]:
test_df_id['quarterly_sales_hl_adjusted'] = test_df_id.apply(lambda x: adjusted(x) , axis=1)

In [120]:
test_df_id['quarterly_sales_hl'].mean() ,test_df_id['quarterly_sales_hl'].median()

(13.372055, 3.397965)

In [121]:
test_df_id['quarterly_sales_hl_adjusted'].mean(),test_df_id['quarterly_sales_hl_adjusted'].median()

(11.990251300011572, 2.6435776948928833)

In [122]:
test_df_id_qrtr = test_df_id.groupby(['ppg_name','unique_poc_id','year','quarter'],
                                     as_index=False).agg({"quarterly_sales_hl":"sum",
                                        'quarterly_sales_hl_adjusted':'sum'})

In [123]:
test_df_id_qrtr.shape

(197348, 6)

In [124]:
submission_df = submission_df.drop(['quarterly_sales_hl'],axis=1) 

In [125]:
submission_df_merge = pd.merge(submission_df,test_df_id_qrtr,
                               on=["ppg_name","unique_poc_id","year","quarter"],
                              how='left')

In [126]:
submission_df_merge.isnull().sum()

year                           0
quarter                        0
ppg_name                       0
unique_poc_id                  0
quarterly_sales_hl             0
quarterly_sales_hl_adjusted    0
dtype: int64

In [129]:
iter_name = 2
os.makedirs(os.path.join(os.getcwd(),"sample_submissions"), exist_ok=True)
submission_df_merge.to_csv(os.path.join(os.getcwd(),"sample_submissions",f'Submission_{iter_name}.csv'),index=False)